In [16]:
from gurobipy import *
from prettytable import PrettyTable
from math import pi
import numpy as np

In [17]:
OPF = Model('3 bus OPF')

In [121]:
#Sets
G = 3 # Generators
B = 3 # Bus
L = 3 #Lines
T = 1
Gen = range(G)
Bus = range(B)
Line = range(L)
Period = range(T)

### Input the susceptance for each transmission line

In [122]:
B1 = {(0,1):{'B':-10 ,'Limit':1000 },
      (0,2):{'B':-8 ,'Limit':1000 },
      (1,2):{'B':-5 ,'Limit':1500 } }

### Generate the graph

In [123]:
y = list(B1.keys())
x=list(map (lambda t: (t[1], t[0]), y))
B2=dict(zip(x,B1.values()))
B = {**B1, **B2}

In [124]:
tuple_list = list(B.keys())
desired_list = [tuple(list(tup)+[t])  for t in Period for tup in tuple_list]
desired_list
vals=[B[(i,j)] for (i,j) in B.keys()]*T
B = dict(zip(desired_list,vals))
B

{(0, 1, 0): {'B': -10, 'Limit': 1000},
 (0, 2, 0): {'B': -8, 'Limit': 1000},
 (1, 2, 0): {'B': -5, 'Limit': 1500},
 (1, 0, 0): {'B': -10, 'Limit': 1000},
 (2, 0, 0): {'B': -8, 'Limit': 1000},
 (2, 1, 0): {'B': -5, 'Limit': 1500}}

In [125]:
L = {(0,0):200, (1,0):550, (2,0):100} # electircal Load (MW)
Sbase = 100 # Power Base (MVA)

In [126]:
GenAt = {Gen[0]:{'a':0.001562 ,'b':7.92 ,'c':561, 'Pmin':150 ,'Pmax':600},
         Gen[1]:{'a':0.001940 ,'b':7.85 ,'c':310, 'Pmin':100 ,'Pmax':400},
         Gen[2]:{'a':0.004820 ,'b':7.97 ,'c':78,  'Pmin':50 , 'Pmax':200}}
GenAt

{0: {'a': 0.001562, 'b': 7.92, 'c': 561, 'Pmin': 150, 'Pmax': 600},
 1: {'a': 0.00194, 'b': 7.85, 'c': 310, 'Pmin': 100, 'Pmax': 400},
 2: {'a': 0.00482, 'b': 7.97, 'c': 78, 'Pmin': 50, 'Pmax': 200}}

In [127]:
Pmin = [(GenAt[i]['Pmin']) for i in Gen]
Pmax = [(GenAt[i]['Pmax']) for i in Gen]
g = [(a,t) for a in Gen for t in Period]
b = [(a,t) for a in Bus for t in Period] 

In [60]:
P = OPF.addVars(g, lb = Pmin, ub = Pmax, name = 'Pgen')
Delta = OPF.addVars(b, lb = -6, ub = 6, name = 'Delta')
Pij = OPF.addVars(B, lb = [-B[(i,j,t)]['Limit'] for (i,j) in B.keys()], 
                  ub = [B[(i,j,t)]['Limit'] for (i,j) in B.keys()], name = 'PF')

In [25]:
PowerFlow = OPF.addConstrs((Pij[(i,j)] == Sbase*(Delta[i]-Delta[j])*(B[(i,j)]['B']) for (i,j) in Pij.keys()), name = 'PowerFlow')
NetBus = OPF.addConstrs( (P[i] - (L[i])  == Pij.sum(i,'*') for i in Gen ), name = 'NetBus')

Delta[0].lb = 0
Delta[0].ub = 0

In [26]:
OF = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)

In [27]:
OPF.setObjective(OF, GRB.MINIMIZE)
OPF.Params.Method=-1
OPF.setParam( 'OutputFlag', True )
OPF.optimize()



status = OPF.status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
or status == GRB.Status.UNBOUNDED:
    OPF.computeIIS()
    OPF.write("model.ilp")
OPF.write("model.lp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 9 rows, 12 columns and 27 nonzeros
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 8e+00]
  QObjective range [3e-03, 1e-02]
  Bounds range     [6e+00, 2e+03]
  RHS range        [1e+02, 6e+02]
Presolve removed 8 rows and 9 columns
Presolve time: 0.01s
Presolved: 1 rows, 3 columns, 3 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.57901300e+06 -1.29551045e+06  2.50e+03 9.93e+02  1.01e+06     0s
   1   8.19941466e+03 -5.67640472e+04  2.50e-03 9.93e-04

In [28]:
OPF.printStats()


Statistics for model 3 bus OPF :
  Linear constraint matrix    : 9 Constrs, 12 Vars, 27 NZs
  Matrix coefficient range    : [ 1, 1000 ]
  Objective coefficient range : [ 7.85, 7.97 ]
  Variable bound range        : [ 6, 1500 ]
  RHS coefficient range       : [ 100, 550 ]


In [29]:
OPF.getVars()

[<gurobi.Var Pgen[0] (value 393.1698369628845)>,
 <gurobi.Var Pgen[1] (value 334.6037552894586)>,
 <gurobi.Var Pgen[2] (value 122.22640774765692)>,
 <gurobi.Var Delta[0] (value 0.0)>,
 <gurobi.Var Delta[1] (value 0.15817759661757372)>,
 <gurobi.Var Delta[2] (value 0.04374030043163849)>,
 <gurobi.Var PF[0,1] (value 158.17759661757373)>,
 <gurobi.Var PF[0,2] (value 34.99224034531079)>,
 <gurobi.Var PF[1,2] (value -57.21864809296767)>,
 <gurobi.Var PF[1,0] (value -158.17759661757373)>,
 <gurobi.Var PF[2,0] (value -34.99224034531079)>,
 <gurobi.Var PF[2,1] (value 57.21864809296771)>]

In [30]:
Pij[(1,2)].lb

-1500.0

In [92]:
tuple_list = [(0,1), (0,2), (1,2)]
time = range(5)

In [93]:
desired_list = [tuple(list(tup)+[t])  for t in time for tup in tuple_list]

In [94]:
desired_list

[(0, 1, 0),
 (0, 2, 0),
 (1, 2, 0),
 (0, 1, 1),
 (0, 2, 1),
 (1, 2, 1),
 (0, 1, 2),
 (0, 2, 2),
 (1, 2, 2),
 (0, 1, 3),
 (0, 2, 3),
 (1, 2, 3),
 (0, 1, 4),
 (0, 2, 4),
 (1, 2, 4)]